### Importing the libararies

In [1]:
import os
import shutil
import pandas as pd
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
full_df=pd.read_csv('lables.csv')

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Classifying into main categories(CHAIR,SOFA,BED,ETC)

In [3]:
full_df['catgory']=full_df['category'].str.split(' ')
full_df['CLASS']=[full_df['catgory'][i][-1] for i in range(len(full_df))]


In [4]:
full_df.head()

,Unnamed: 0,paths,url,manufactured by,sold by,size/weight,color,materials,assembly required,category,...,collection,amount,currency,commercial-grade,size,weight,designer,brand,catgory,CLASS
0,0,/home/dima/ssd/furniture_data/pictures/5e91448...,https://www.houzz.com/products/two-door-cabine...,Coast to Coast Accents,BisonOffice,"W 24"" / D 40"" / H 49.5"" / 110.62 lb.",NaN,NaN,No,Accent And Storage Benches,...,NaN,735.0,USD,NaN,NaN,NaN,NaN,NaN,"[Accent, And, Storage, Benches]",Benches
1,1,/home/dima/ssd/furniture_data/pictures/d591fd0...,https://www.houzz.com/products/talia-industria...,Armen Living,Luxx Kitchen and Bath,"W 24"" / D 24"" / H 30.5"" / 58.5 lb.",Pine,NaN,No,Indoor Pub And Bistro Tables,...,NaN,272.0,USD,NaN,NaN,NaN,NaN,NaN,"[Indoor, Pub, And, Bistro, Tables]",Tables
2,2,/home/dima/ssd/furniture_data/pictures/8d91534...,https://www.houzz.com/products/danbury-chair-1...,Fairfield Chair,Houzz,"W 26"" / D 25"" / H 33"" / 50 lb.",1175 Denim Leather,Beech,No,Dining Chairs,...,Danbury,1249.0,USD,NaN,NaN,NaN,NaN,NaN,"[Dining, Chairs]",Chairs
3,3,/home/dima/ssd/furniture_data/pictures/b831bf6...,https://www.houzz.com/products/nico-rustic-ind...,Baxton Studio,Skyline Decor,"W 59.8"" / D 36.02"" / H 36.02"" / 81.9 lb.",NaN,MDF/Metal,Yes,Desks And Hutches,...,NaN,1082.0,USD,Yes,NaN,NaN,NaN,NaN,"[Desks, And, Hutches]",Hutches
4,4,/home/dima/ssd/furniture_data/pictures/9bc1ae0...,https://www.houzz.com/products/love-my-dog-win...,American Glassware,Houzz,"W 3.25"" / D 3.25"" / H 9"" / 15 oz.",NaN,glass,No,Wine Glasses,...,Bestsellers,44.0,USD,NaN,NaN,NaN,NaN,NaN,"[Wine, Glasses]",Glasses


## Keeping top 50 colors ,we still have almost 40% data

In [4]:
selected_colors = (full_df.groupby('color')['paths'].count().sort_values(ascending=False)[:50]).index

In [5]:
mapping_df = full_df[full_df['color'].isin(selected_colors)]
len(mapping_df)

231773

In [12]:
mapping_df.head()

,Unnamed: 0,paths,url,manufactured by,sold by,size/weight,color,materials,assembly required,category,...,collection,amount,currency,commercial-grade,size,weight,designer,brand,catgory,CLASS
2,2,/home/dima/ssd/furniture_data/pictures/8d91534...,https://www.houzz.com/products/danbury-chair-1...,Fairfield Chair,Houzz,"W 26"" / D 25"" / H 33"" / 50 lb.",1175 Denim Leather,Beech,No,Dining Chairs,...,Danbury,1249.0,USD,NaN,NaN,NaN,NaN,NaN,"[Dining, Chairs]",Chairs
5,5,/home/dima/ssd/furniture_data/pictures/239121f...,https://www.houzz.com/products/2-in-x-2-in-gla...,Abolos,Abolos,"W 12"" / D 12"" / H 0.25"" / 2.36 lb.",Silver,Glass and Metal,Yes,Mosaic Tile,...,Enchanted Metals,12.0,USD,NaN,NaN,NaN,NaN,NaN,"[Mosaic, Tile]",Tile
11,11,/home/dima/ssd/furniture_data/pictures/592156f...,https://www.houzz.com/products/lorell-mobile-f...,lorell_biehler,BisonOffice,"W 15"" / D 22.9"" / H 27.8"" / 84 lb.",Black,Steel,No,Filing Cabinets,...,NaN,210.0,USD,NaN,NaN,NaN,NaN,NaN,"[Filing, Cabinets]",Cabinets
23,23,/home/dima/ssd/furniture_data/pictures/84711ac...,https://www.houzz.com/products/angela-full-que...,LexMod,Uber Bazaar,"W 61.5"" / D 3"" / H 22.5"" / 20 lb.",Charcoal,"MDF, LVL, Plywood, Fabric Material: 100% Polyster",Yes,Headboards,...,NaN,271.0,USD,Yes,NaN,NaN,NaN,NaN,[Headboards],Headboards
27,27,/home/dima/ssd/furniture_data/pictures/d3b10b6...,https://www.houzz.com/products/caliber-cabinet...,Amerock Hardware,Houzz,"W 0.5"" / D 1.25"" / H 8.44"" / 6.24 oz.",Polished Chrome,Zinc,No,Cabinet And Drawer Handle Pulls,...,NaN,24.0,USD,Yes,NaN,NaN,NaN,NaN,"[Cabinet, And, Drawer, Handle, Pulls]",Pulls


### Making COLOR-AND-CATEGORY combined label

In [6]:
mapping_df['COLORCATG']=mapping_df['color']+','+mapping_df['CLASS']

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


## SO keeping first 40 ,we obtain quite a balanced data

In [7]:
mapping_df['COLORCATG'].value_counts()[:40]

Black,Chairs                    5769
Gray,Chairs                     3764
White,Consoles                  3733
White,Chairs                    3558
Gray,Beds                       3014
Black,Stools                    2638
White,Beds                      2359
Brown,Chairs                    2306
White,Tables                    2300
Gray,Sets                       2224
1175 Denim Leather,Chairs       2197
1179 Taupe Leather,Chairs       2185
1163 Saddle Leather,Chairs      2185
1168 Truffle Leather,Chairs     2182
1178 Cognac Leather,Chairs      2179
1179 Saddle Leather,Chairs      2179
8796 Natural Fabric,Chairs      2155
9508 Sand Fabric,Chairs         2152
9508 Oasis Fabric,Chairs        2152
9508 Hazelnut Fabric,Chairs     2152
8703 Bamboo Fabric,Chairs       2146
8794 Platinum Fabric,Chairs     2144
9953 Midnight Fabric,Chairs     2139
8704 Dune Fabric,Chairs         2136
9508 Smoke Fabric,Chairs        2136
8703 Alabaster Fabric,Chairs    2131
Brown,Tables                    2129
G

In [8]:
selected_categories = (mapping_df.groupby('COLORCATG')['paths'].count().sort_values(ascending=False)[:40]).index
mapping_dff = mapping_df[mapping_df['COLORCATG'].isin(selected_categories)]
mapping_dff

,Unnamed: 0,paths,url,manufactured by,sold by,size/weight,color,materials,assembly required,category,...,amount,currency,commercial-grade,size,weight,designer,brand,catgory,CLASS,COLORCATG
2,2,/home/dima/ssd/furniture_data/pictures/8d91534...,https://www.houzz.com/products/danbury-chair-1...,Fairfield Chair,Houzz,"W 26"" / D 25"" / H 33"" / 50 lb.",1175 Denim Leather,Beech,No,Dining Chairs,...,1249.0,USD,NaN,NaN,NaN,NaN,NaN,"[Dining, Chairs]",Chairs,"1175 Denim Leather,Chairs"
54,54,/home/dima/ssd/furniture_data/pictures/2281401...,https://www.houzz.com/products/60-x-24-kobe-mo...,Regency,BisonOffice,"W 62"" / D 26"" / H 38"" / 106 lb.",Black,NaN,Yes,Kids Desks And Desk Sets,...,970.0,USD,NaN,NaN,NaN,NaN,NaN,"[Kids, Desks, And, Desk, Sets]",Sets,"Black,Sets"
57,57,/home/dima/ssd/furniture_data/pictures/ef31fa2...,https://www.houzz.com/products/palmer-arm-chai...,Fairfield Chair,Houzz,"W 24"" / D 24"" / H 38.5"" / 24 lb.",9953 Midnight Fabric,Beech,No,Office Chairs,...,938.0,USD,NaN,NaN,NaN,NaN,NaN,"[Office, Chairs]",Chairs,"9953 Midnight Fabric,Chairs"
66,66,/home/dima/ssd/furniture_data/pictures/4101cf8...,https://www.houzz.com/products/alma-edison-fre...,ALMA VANITIES,Alma Premium Vanities,"W 35.5"" / D 19.7"" / H 35.5"" / 160 lb.",White,Engineering wood,No,Bathroom Vanities And Sink Consoles,...,952.0,USD,NaN,NaN,NaN,NaN,NaN,"[Bathroom, Vanities, And, Sink, Consoles]",Consoles,"White,Consoles"
67,67,/home/dima/ssd/furniture_data/pictures/20b1646...,https://www.houzz.com/products/77console-table...,Noble Origins Home,Noble Origins Home,"W 77"" / D 14"" / H 29"" / 265 lb.",Black,Glass,No,Console Tables,...,5291.0,USD,NaN,NaN,NaN,NaN,NaN,"[Console, Tables]",Tables,"Black,Tables"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730128,730128,/home/dima/ssd/furniture_data/pictures/1341bc1...,https://www.houzz.com/products/zero-gravity-ch...,Brawbuy,Brawbuy Deals,"W 35.4"" / D 43.3"" / H 25.6"" / 16.5 lb.",Blue,NaN,No,Outdoor Lounge Chairs,...,77.0,USD,NaN,NaN,NaN,NaN,NaN,"[Outdoor, Lounge, Chairs]",Chairs,"Blue,Chairs"
730129,730129,/home/dima/ssd/furniture_data/pictures/27a168f...,https://www.houzz.com/products/flash-furniture...,Flash Furniture,BisonOffice,NaN,Black,NaN,Yes,Bar Stools And Counter Stools,...,117.6,USD,NaN,"W 16"" / D 19"" / H 44""",NaN,NaN,NaN,"[Bar, Stools, And, Counter, Stools]",Stools,"Black,Stools"
730137,730137,/home/dima/ssd/furniture_data/pictures/8911738...,https://www.houzz.com/products/furniture-of-am...,Furniture of America E-Commerce by Enitial Lab,Homesquare,"W 24.5"" / D 26"" / H 38"" / 50 lb.",Brown,Wood,Yes,Gaming Chairs,...,590.0,USD,NaN,NaN,NaN,NaN,NaN,"[Gaming, Chairs]",Chairs,"Brown,Chairs"
730139,730139,/home/dima/ssd/furniture_data/pictures/ed41acb...,https://www.houzz.com/products/max-arm-chair-1...,Fairfield Chair,Houzz,"W 25.5"" / D 26"" / H 41.5"" / 45 lb.",1168 Truffle Leather,Beech,No,Dining Chairs,...,1121.0,USD,NaN,NaN,NaN,NaN,NaN,"[Dining, Chairs]",Chairs,"1168 Truffle Leather,Chairs"


In [9]:
required_df=mapping_dff[['paths','COLORCATG']].copy()

In [10]:
required_df

,paths,COLORCATG
2,/home/dima/ssd/furniture_data/pictures/8d91534...,"1175 Denim Leather,Chairs"
54,/home/dima/ssd/furniture_data/pictures/2281401...,"Black,Sets"
57,/home/dima/ssd/furniture_data/pictures/ef31fa2...,"9953 Midnight Fabric,Chairs"
66,/home/dima/ssd/furniture_data/pictures/4101cf8...,"White,Consoles"
67,/home/dima/ssd/furniture_data/pictures/20b1646...,"Black,Tables"
...,...,...
730128,/home/dima/ssd/furniture_data/pictures/1341bc1...,"Blue,Chairs"
730129,/home/dima/ssd/furniture_data/pictures/27a168f...,"Black,Stools"
730137,/home/dima/ssd/furniture_data/pictures/8911738...,"Brown,Chairs"
730139,/home/dima/ssd/furniture_data/pictures/ed41acb...,"1168 Truffle Leather,Chairs"


* Only keeping the final path
### Also, I am keeping the real index, so that later on we can account for common

In [11]:
required_df['paths']=required_df['paths'].str.split('/')
required_df['paths']=[required_df['paths'][i][6] for i in (required_df.index)]
required_df

,paths,COLORCATG
2,8d9153490f485614_4-9191.jpg,"1175 Denim Leather,Chairs"
54,2281401d0dca7b8e_4-2531.jpg,"Black,Sets"
57,ef31fa210f485fc6_4-0799.jpg,"9953 Midnight Fabric,Chairs"
66,4101cf8d0df28d2a_4-8635.jpg,"White,Consoles"
67,20b164690dd72538_4-2566.jpg,"Black,Tables"
...,...,...
730128,1341bc1f0aa7e9c1_4-3795.jpg,"Blue,Chairs"
730129,27a168fc077fa7f8_4-6738.jpg,"Black,Stools"
730137,891173830c18438c_4-3013.jpg,"Brown,Chairs"
730139,ed41acb50f485cde_4-0722.jpg,"1168 Truffle Leather,Chairs"


In [12]:
required_df.to_csv('useful-csvs/color-categr.csv')

## Now checking STYLE, and MATERIAL

##  So, may be later we can use MANUFACTURED_BY ,as we got some dense categories in it, but there are not in COLLECTION

### The category SOLD BY , is highly biased towards one class, so may be, this will negatively affect our training

In [27]:
full_df['sold by'].value_counts()

Houzz                          375003
Homesquare                      29637
GDFStudio                       25987
BisonOffice                     16966
Apt2B                           14894
                                ...  
L.V. Harkness and Company           1
T-Trove Asian Decor                 1
Blue Ridge Log Works                1
Brio Stock                          1
Home and Patio Decor Center         1
Name: sold by, Length: 963, dtype: int64

In [15]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730152 entries, 0 to 730151
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         730152 non-null  int64  
 1   paths              730152 non-null  object 
 2   url                730152 non-null  object 
 3   manufactured by    727784 non-null  object 
 4   sold by            730152 non-null  object 
 5   size/weight        677787 non-null  object 
 6   color              540043 non-null  object 
 7   materials          638680 non-null  object 
 8   assembly required  677490 non-null  object 
 9   category           730152 non-null  object 
 10  style              716177 non-null  object 
 11  collection         215855 non-null  object 
 12  amount             729407 non-null  float64
 13  currency           730152 non-null  object 
 14  commercial-grade   156067 non-null  object 
 15  size               18664 non-null   object 
 16  we

In [ ]:
df['paths']=df['paths'].str.split('/')
df['paths']=[df['paths'][i][6] for i in (df.index)]
required_df

In [33]:
refined_csv=df.drop(labels=['Unnamed: 0','Unnamed: 0','currency'],axis=1)
refined_csv.to_csv('useful-csvs/refined-full.csv')

* SO, lets categorize with-respect-to STYLE-CATEGORY

In [34]:
df=pd.read_csv('useful-csvs/refined-full.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,paths,url,manufactured by,sold by,size/weight,color,materials,assembly required,category,style,amount,catgory,CLASS
0,0,0,5e9144890d5d975b_4-1795.jpg,https://www.houzz.com/products/two-door-cabine...,Coast to Coast Accents,BisonOffice,"W 24"" / D 40"" / H 49.5"" / 110.62 lb.",NaN,NaN,No,Accent And Storage Benches,Transitional,735.0,"['Accent', 'And', 'Storage', 'Benches']",Benches
1,1,1,d591fd080bbbd19e_4-5553.jpg,https://www.houzz.com/products/talia-industria...,Armen Living,Luxx Kitchen and Bath,"W 24"" / D 24"" / H 30.5"" / 58.5 lb.",Pine,NaN,No,Indoor Pub And Bistro Tables,Industrial,272.0,"['Indoor', 'Pub', 'And', 'Bistro', 'Tables']",Tables
2,2,2,8d9153490f485614_4-9191.jpg,https://www.houzz.com/products/danbury-chair-1...,Fairfield Chair,Houzz,"W 26"" / D 25"" / H 33"" / 50 lb.",1175 Denim Leather,Beech,No,Dining Chairs,Transitional,1249.0,"['Dining', 'Chairs']",Chairs
3,3,3,b831bf6c0eb29e50_4-6346.jpg,https://www.houzz.com/products/nico-rustic-ind...,Baxton Studio,Skyline Decor,"W 59.8"" / D 36.02"" / H 36.02"" / 81.9 lb.",NaN,MDF/Metal,Yes,Desks And Hutches,Industrial,1082.0,"['Desks', 'And', 'Hutches']",Hutches
4,4,4,9bc1ae0207d84cf9_4-0688.jpg,https://www.houzz.com/products/love-my-dog-win...,American Glassware,Houzz,"W 3.25"" / D 3.25"" / H 9"" / 15 oz.",NaN,glass,No,Wine Glasses,Contemporary,44.0,"['Wine', 'Glasses']",Glasses


In [35]:
df['STYLEmATERIAL']=df['style']+','+df['materials']

In [38]:
df['STYLEmATERIAL'].value_counts()[:10]

Transitional,Beech                                                        28561
Contemporary,Stainless Steel                                              15707
Transitional,Wood                                                         14356
Tropical,Wicker                                                            9689
Traditional,Beech                                                          6389
Contemporary,Glass                                                         5435
Modern,HDPE (High Density Polyethylene) Wicker                             5269
Transitional,Solid Wood Frame, Hypoallergenic Fabrics, Foam Cushioning     5150
Transitional,Manufactured Wood, Velvet                                     5020
Contemporary,Metal                                                         4991
Name: STYLEmATERIAL, dtype: int64

In [39]:
df['STYLEcOLOR']=df['style']+','+df['color']

In [41]:
df['STYLEcOLOR'].value_counts()[:10]

Contemporary,Black              12159
Contemporary,White              10940
Transitional,Gray                8276
Transitional,White               8075
Contemporary,Gray                6139
Transitional,Black               5223
Transitional,Brown               3496
Contemporary,Red                 3376
Contemporary,Stainless Steel     3187
Contemporary,Brown               3147
Name: STYLEcOLOR, dtype: int64

In [42]:
selected_categories = (df.groupby('STYLEcOLOR')['paths'].count().sort_values(ascending=False)[:40]).index
style_color_df =df[df['STYLEcOLOR'].isin(selected_categories)]


In [43]:
len(style_color_df)

123806

In [44]:
style_color_df['STYLEcOLOR'].value_counts()

Contemporary,Black                    12159
Contemporary,White                    10940
Transitional,Gray                      8276
Transitional,White                     8075
Contemporary,Gray                      6139
Transitional,Black                     5223
Transitional,Brown                     3496
Contemporary,Red                       3376
Contemporary,Stainless Steel           3187
Contemporary,Brown                     3147
Contemporary,Blue                      3094
Transitional,Espresso                  2736
Midcentury,Gray                        2660
Transitional,Beige                     2449
Contemporary,Green                     2337
Midcentury,Black                       2253
Contemporary,Navy                      2071
Midcentury,White                       1939
Contemporary,Chrome                    1917
Transitional,1175 Denim Leather        1905
Transitional,1179 Taupe Leather        1893
Transitional,1163 Saddle Leather       1893
Transitional,1168 Truffle Leathe

### SO, we have an acceptable range from max12000 to min 1500

In [46]:
required_df=style_color_df[['paths','STYLEcOLOR']]

In [48]:
required_df.to_csv('useful-csvs/style-color.csv')

### Let's see if we can get more complex attributes

In [50]:
df['CatgMatrColr']=df['catgory']+','+df['materials']+','+df['color']

In [55]:
df['CatgMatrColr'].value_counts()[:50]

['Kitchen', 'Faucets'],Brass,Polished Chrome                                                 1367
['Kitchen', 'Sinks'],Stainless Steel,Stainless Steel                                         1229
['Office', 'Chairs'],Beech,1163 Saddle Leather                                               1175
['Office', 'Chairs'],Beech,1175 Denim Leather                                                1169
['Office', 'Chairs'],Beech,1178 Cognac Leather                                               1169
['Office', 'Chairs'],Beech,1179 Saddle Leather                                               1169
['Office', 'Chairs'],Beech,1168 Truffle Leather                                              1167
['Office', 'Chairs'],Beech,1179 Taupe Leather                                                1163
['Office', 'Chairs'],Beech,9508 Hazelnut Fabric                                              1130
['Office', 'Chairs'],Beech,9508 Oasis Fabric                                                 1130
['Office', 'Chairs']

* This is good, we have 50 categories with variation from 1300 to 550

In [56]:
required_df=df[['paths','CatgMatrColr']]

In [62]:
selected_categories = (required_df.groupby('CatgMatrColr')['paths'].count().sort_values(ascending=False)[:50]).index
required_df =required_df[required_df['CatgMatrColr'].isin(selected_categories)]


In [63]:
len(required_df)

47093

In [66]:
required_df.to_csv('useful-csvs/catg-matr-color.csv')